In [1]:
# !pip install tensorflow-gpu==2.3
# !pip install sklearn
# !pip install matplotlib

In [2]:
import numpy as np
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
import os
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import time

In [3]:
import tensorflow
import tensorflow as tf

In [4]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [5]:
file_dir1 = 'C:/Users/BVer/are_you_ok/preprocessing'

In [30]:
raw_data = np.load(file_dir1 + '/p_video/' + 'raw_kicking_k1.mp4_.npy')

In [40]:
raw_data = np.load(file_dir1 + '/p_video/' + 'raw_test_video1.mp4_.npy')

In [8]:
raw_data.shape

(2489,)

In [9]:
from tensorflow.keras.models import load_model

right_arm_model = load_model('./models/right_arm_model.h5')
left_arm_model = load_model('./models/left_arm_model.h5')
right_leg_model = load_model('./models/right_leg_model.h5')
left_leg_model = load_model('./models/left_leg_model.h5')

## openpose 모델로드 및 gpu 설정

In [3]:
device = "gpu" # please change it to "gpu" if the model needs to be run on cuda.

protoFile = "pose_deploy_linevec.prototxt"
weightsFile = "pose_iter_440000.caffemodel"
nPoints = 18
# COCO Output Format
keypointsMapping = ['Nose', 'Neck', 'R-Sho', 'R-Elb', 'R-Wr', 'L-Sho', 
                    'L-Elb', 'L-Wr', 'R-Hip', 'R-Knee', 'R-Ank', 'L-Hip', 
                    'L-Knee', 'L-Ank', 'R-Eye', 'L-Eye', 'R-Ear', 'L-Ear']

POSE_PAIRS = [[1,2], [1,5], [2,3], [3,4], [5,6], [6,7],
              [1,8], [8,9], [9,10], [1,11], [11,12], [12,13],
              [1,0], [0,14], [14,16], [0,15], [15,17],
              [2,17], [5,16] ]

# index of pafs correspoding to the POSE_PAIRS
# e.g for POSE_PAIR(1,2), the PAFs are located at indices (31,32) of output, Similarly, (1,5) -> (39,40) and so on.
mapIdx = [[31,32], [39,40], [33,34], [35,36], [41,42], [43,44], 
          [19,20], [21,22], [23,24], [25,26], [27,28], [29,30], 
          [47,48], [49,50], [53,54], [51,52], [55,56], 
          [37,38], [45,46]]

colors = [ [0,100,255], [0,100,255], [0,255,255], [0,100,255], [0,255,255], [0,100,255],
         [0,255,0], [255,200,100], [255,0,255], [0,255,0], [255,200,100], [255,0,255],
         [0,0,255], [255,0,0], [200,200,0], [255,0,0], [200,200,0], [0,0,0]]

net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

if device == "cpu":
    net.setPreferableBackend(cv2.dnn.DNN_TARGET_CPU)
    print("Using CPU device")
elif device == "gpu":
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
    print("Using GPU device")

Using GPU device


## openpose 함수

In [4]:
# Find the Keypoints using Non Maximum Suppression on the Confidence Map
def getKeypoints(probMap, threshold=0.1):
    
    mapSmooth = cv2.GaussianBlur(probMap,(3,3),0,0)

    mapMask = np.uint8(mapSmooth>threshold)
    keypoints = []
    
    #find the blobs
    contours, _ = cv2.findContours(mapMask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    #for each blob find the maxima
    for cnt in contours:
        blobMask = np.zeros(mapMask.shape)
        blobMask = cv2.fillConvexPoly(blobMask, cnt, 1)
        maskedProbMap = mapSmooth * blobMask
        _, maxVal, _, maxLoc = cv2.minMaxLoc(maskedProbMap)
        keypoints.append(maxLoc + (probMap[maxLoc[1], maxLoc[0]],))

    return keypoints

# Find valid connections between the different joints of a all persons present
def getValidPairs(output, frameWidth, frameHeight, detected_keypoints):
    valid_pairs = []
    invalid_pairs = []
    n_interp_samples = 10
    paf_score_th = 0.1
    conf_th = 0.7
    # loop for every POSE_PAIR
    for k in range(len(mapIdx)):
        # A->B constitute a limb
        pafA = output[0, mapIdx[k][0], :, :]
        pafB = output[0, mapIdx[k][1], :, :]
        pafA = cv2.resize(pafA, (frameWidth, frameHeight))
        pafB = cv2.resize(pafB, (frameWidth, frameHeight))

        # Find the keypoints for the first and second limb
        candA = detected_keypoints[POSE_PAIRS[k][0]]
        candB = detected_keypoints[POSE_PAIRS[k][1]]
        nA = len(candA)
        nB = len(candB)

        # If keypoints for the joint-pair is detected
        # check every joint in candA with every joint in candB 
        # Calculate the distance vector between the two joints
        # Find the PAF values at a set of interpolated points between the joints
        # Use the above formula to compute a score to mark the connection valid
        
        if( nA != 0 and nB != 0):
            valid_pair = np.zeros((0,3))
            for i in range(nA):
                max_j=-1
                maxScore = -1
                found = 0
                for j in range(nB):
                    # Find d_ij
                    d_ij = np.subtract(candB[j][:2], candA[i][:2])
                    norm = np.linalg.norm(d_ij)
                    if norm:
                        d_ij = d_ij / norm
                    else:
                        continue
                    # Find p(u)
                    interp_coord = list(zip(np.linspace(candA[i][0], candB[j][0], num=n_interp_samples),
                                            np.linspace(candA[i][1], candB[j][1], num=n_interp_samples)))
                    # Find L(p(u))
                    paf_interp = []
                    for k in range(len(interp_coord)):
                        paf_interp.append([pafA[int(round(interp_coord[k][1])), int(round(interp_coord[k][0]))],
                                           pafB[int(round(interp_coord[k][1])), int(round(interp_coord[k][0]))] ]) 
                    # Find E
                    paf_scores = np.dot(paf_interp, d_ij)
                    avg_paf_score = sum(paf_scores)/len(paf_scores)
                    
                    # Check if the connection is valid
                    # If the fraction of interpolated vectors aligned with PAF is higher then threshold -> Valid Pair  
                    if ( len(np.where(paf_scores > paf_score_th)[0]) / n_interp_samples ) > conf_th :
                        if avg_paf_score > maxScore:
                            max_j = j
                            maxScore = avg_paf_score
                            found = 1
                # Append the connection to the list
                if found:            
                    valid_pair = np.append(valid_pair, [[candA[i][3], candB[max_j][3], maxScore]], axis=0)

            # Append the detected connections to the global list
            valid_pairs.append(valid_pair)
        else: # If no keypoints are detected
#             print("No Connection : k = {}".format(k))
            invalid_pairs.append(k)
            valid_pairs.append([])
#     print(valid_pairs)
    return valid_pairs, invalid_pairs

# This function creates a list of keypoints belonging to each person
# For each detected valid pair, it assigns the joint(s) to a person
# It finds the person and index at which the joint should be added. This can be done since we have an id for each joint
def getPersonwiseKeypoints(valid_pairs, invalid_pairs, keypoints_list):
    # the last number in each row is the overall score 
    personwiseKeypoints = -1 * np.ones((0, 19))

    for k in range(len(mapIdx)):
        if k not in invalid_pairs:
            partAs = valid_pairs[k][:,0]
            partBs = valid_pairs[k][:,1]
            indexA, indexB = np.array(POSE_PAIRS[k])

            for i in range(len(valid_pairs[k])): 
                found = 0
                person_idx = -1
                for j in range(len(personwiseKeypoints)):
                    if personwiseKeypoints[j][indexA] == partAs[i]:
                        person_idx = j
                        found = 1
                        break

                if found:
                    personwiseKeypoints[person_idx][indexB] = partBs[i]
                    personwiseKeypoints[person_idx][-1] += keypoints_list[partBs[i].astype(int), 2] + valid_pairs[k][i][2]

                # if find no partA in the subset, create a new subset
                elif not found and k < 17:
                    row = -1 * np.ones(19)
                    row[indexA] = partAs[i]
                    row[indexB] = partBs[i]
                    # add the keypoint_scores for the two keypoints and the paf_score 
                    row[-1] = sum(keypoints_list[valid_pairs[k][i,:2].astype(int), 2]) + valid_pairs[k][i][2]
                    personwiseKeypoints = np.vstack([personwiseKeypoints, row])
    return personwiseKeypoints

# 영상 처리

In [2]:
def pred_multiple_onvideo(video_path, filename):
    if video_path == 'cam':
        cap = cv2.VideoCapture(int(filename))
    else:
        cap = cv2.VideoCapture(video_path)
        
    data = []
    position = []
    last_position = []
    final_position_index = -1
    detected_humans = [] #[human][index, part_seq, part_nan_count, part_action, count]
    
    ret, image1 = cap.read()
    while cap.isOpened():
        t0 = cv2.getTickCount()
        if ret == False:
            break
            
        frameWidth = image1.shape[1]
        frameHeight = image1.shape[0]

        # Fix the input Height and get the width according to the Aspect Ratio
        inHeight = 368 #인식에 문제가 있다면 이 값을 크게 조절하세요 (max = videofile height)
        inWidth = int((inHeight/frameHeight)*frameWidth)
        inpBlob = cv2.dnn.blobFromImage(image1, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False)

        net.setInput(inpBlob)
        output = net.forward()

        i = 0
        probMap = output[0, i, :, :]
        probMap = cv2.resize(probMap, (frameWidth, frameHeight))

        detected_keypoints = []
        keypoints_list = np.zeros((0,3))
        keypoint_id = 0
        threshold = 0.1

        for part in range(nPoints):
            probMap = output[0,part,:,:]
            probMap = cv2.resize(probMap, (image1.shape[1], image1.shape[0]))
            keypoints = getKeypoints(probMap, threshold)
            keypoints_with_id = []
            for i in range(len(keypoints)):
                keypoints_with_id.append(keypoints[i] + (keypoint_id,))
                keypoints_list = np.vstack([keypoints_list, keypoints[i]])
                keypoint_id += 1

            detected_keypoints.append(keypoints_with_id)

        valid_pairs, invalid_pairs = getValidPairs(output, frameWidth, frameHeight, detected_keypoints)
        personwiseKeypoints = getPersonwiseKeypoints(valid_pairs, invalid_pairs, keypoints_list)
        
        
        #사람 구분
        detected_Obj = []
        last_position += position
        position = []
        keylist = np.empty((len(personwiseKeypoints), 13, 2))

        for i in range(1, 14):
            for n in range(len(personwiseKeypoints)):
                index = personwiseKeypoints[n][np.array(POSE_PAIRS[i])]
                if -1 in index:
                    continue
                A = np.int32(keypoints_list[personwiseKeypoints[n][i].astype(int)])
                keylist[n][i - 1] = A[:-1] #keylist는 목부터 발까지 좌표 값만 저장됨

        for n in range(len(personwiseKeypoints)):
            short_distance = 10000
            if int(keylist[n][0][0]) == 0:
                continue
            else:
                #여기서 벡터값 계산
                joint = np.empty((13, 2))
                for j in range(13):
                    try:
                        joint[j] = [keylist[n][j][0], keylist[n][j][1]]
                    except:
                        pass

                # Compute angles between joints
                v1 = joint[[0, 1, 2, 0, 4, 5, 0, 7, 8, 0, 10, 11], :2] # Parent joint
                v2 = joint[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], :2] # Child joint
                v = v2 - v1 # [12, 2]
                # Normalize v
                v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                # Get angle using arcos of dot product
                angle = np.arccos(np.einsum('nt,nt->n',
                    v[[0, 1, 3, 4, 6, 7, 9, 10],:], 
                    v[[1, 2, 4, 5, 7, 8, 10, 11],:]))

                angle = np.degrees(angle) # Convert radian to degree

                angle_label = np.array([angle], dtype=np.float32)

                count = -1
                last_p_index = -1
                short_p_index = -1
                for last_p in range(len(last_position)):
                    temp_x = keylist[n][0][0] - last_position[last_p][0]
                    temp_y = keylist[n][0][1] - last_position[last_p][1]
                    temp_distance = temp_x * temp_x + temp_y * temp_y
                    if temp_distance < short_distance:
                        short_distance = temp_distance
                        last_p_index = last_p
                        position_index = last_position[last_p][2]
                        
                if last_p_index == -1: #전 프레임에 같은 사람이 없을때
                    final_position_index+=1
                    position.append([keylist[n][0][0], keylist[n][0][1], final_position_index, 0])
                    
                    for detect_seq in detected_humans: #사람이 전 프레임에 있던 사람인지 확인
                        if final_position_index == detect_seq[0]: #해당 인덱스의 사람이 존재하는지 검사
                            detected_humans.append([final_position_index, [], 0, 0, [], 0, 0, [], 0, 0, [], 0, 0])
                            pred_raw_part(angle_label, detected_humans[-1])

                else: #전 프레임에 같은 사람이 있을때
                    for detect_seq in detected_humans: #사람이 전 프레임에 있던 사람인지 확인
                        if position_index == detect_seq[0]: #해당 인덱스의 사람이 존재하는지 검사
                            position.append([keylist[n][0][0], keylist[n][0][1], position_index, 0])
                            del last_position[last_p_index]
                            pred_raw_part(angle_label, detect_seq)        

        del_list = []         
        for gone in range(len(last_position)):
            last_position[gone][3] += 1
            if last_position[gone][3] > 10:
                del_list.append(last_position[gone])
                

        if len(del_list) > 0:
            for del_position in range(len(del_list)):
                for del_index in detected_humans:
                    if del_index[0] == del_list[del_position]:
                        last_position.remove(del_list[del_position])
                        detected_humans.remove(del_index)
                        break

# 예측

In [47]:
def pred_raw_part(object_n, human_seq):
    object_n = object_n[:-1]
    seq_length = 30

    actions = ['assult', 'normal']
    
    nan_skip_count = 10 #오차 허용 프레임
    
    for process_count in range(4):
        human_seq[process_count * 3 + 2] = take_not_nan(object_n[process_count * 2], object_n[process_count * 2 + 1], human_seq[process_count * 3 + 1], human_seq[process_count * 3 + 2])
        
    for process_count in range(4):
        if human_seq[process_count * 3 + 2] == 15:
            print("cleared seq")
            human_seq[process_count * 3 + 1] = []
            human_seq[process_count * 3 + 3] = 0
            
    for process_count, model in enumerate([right_arm_model, left_arm_model, right_leg_model, left_leg_model]):
        human_seq[process_count * 3 + 3] = pred_part(human_seq[process_count * 3 + 1], human_seq[process_count * 3 + 2], model, human_seq[process_count * 3 + 3])
        
def take_not_nan(key1, key2, part_seq, part_nan):
    if not np.isnan(key1) and not np.isnan(key2):
        part_seq.append([key1, key2])
        return 0
    else:
        part_nan += 1
        if part_nan > 10: #오차 허용 프레임
            print(1111)
            return 15
        else:
            return part_nan
        
def pred_part(part_seq, part_nan, part_model, part_action):
    seq_length = 30
    if len(part_seq) > seq_length and part_nan == 0:
        input_data = np.expand_dims(np.array(part_seq[-seq_length:], dtype=np.float32), axis=0)
        y_pred = part_model.predict(input_data).squeeze()

        i_pred = int(np.argmax(y_pred))
        conf = y_pred[i_pred]
        
        print(conf, i_pred)

        if conf > 0.5 and i_pred == 0: #신뢰도
            part_action += 1
            print(part_action)
            if part_action > 2:
                print('detect  assult ', conf)
            return part_action
        else:
            return 0
    return part_action

# 실행

In [ ]:
now = '/test_video1.mp4'
root_dir = 'C:/Users/BVer/are_you_ok'
video_dir = root_dir + '/datavideo' + now

for i in video_name:
    video_path = video_dir + i
    pred_multiple_onvideo(video_path, i)
    print(video_path, complete)

print('----- finish -----')

preprocessing start
test_video1.mp4
took 6.291913 seconds.
took 6.092342 seconds.
took 6.585195 seconds.
took 8.053255 seconds.
took 7.932958 seconds.
took 7.938286 seconds.
took 7.997549 seconds.
took 7.843977 seconds.
took 7.878574 seconds.
took 8.053193 seconds.
took 9.308623 seconds.
took 8.572394 seconds.
took 8.592335 seconds.
took 9.314547 seconds.
took 10.484539 seconds.
took 8.452636 seconds.
took 8.836285 seconds.
took 8.222702 seconds.
took 7.821009 seconds.
took 7.876937 seconds.
took 7.860767 seconds.
took 8.004947 seconds.
took 8.112739 seconds.
took 8.157218 seconds.
took 8.094727 seconds.
took 8.119388 seconds.
took 7.895737 seconds.
took 7.960257 seconds.
took 8.511864 seconds.
took 8.474245 seconds.
took 8.091807 seconds.
took 7.883660 seconds.
took 7.833548 seconds.
took 7.889282 seconds.
took 8.287921 seconds.
took 8.068801 seconds.
took 8.050977 seconds.
took 8.259074 seconds.
took 7.983337 seconds.
took 7.812114 seconds.
took 7.656246 seconds.
took 7.726842 second

c:\users\bver\miniconda3\envs\ayo\lib\site-packages\numpy\linalg\linalg.py:2506: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real


took 8.056472 seconds.
took 8.056846 seconds.
took 8.105165 seconds.
took 8.172928 seconds.
took 8.079746 seconds.
took 8.239762 seconds.
took 8.499470 seconds.
took 8.640130 seconds.
took 8.263513 seconds.
took 8.199116 seconds.
took 8.162834 seconds.
took 8.165033 seconds.
took 8.136847 seconds.
took 8.118890 seconds.
took 8.150429 seconds.


c:\users\bver\miniconda3\envs\ayo\lib\site-packages\ipykernel_launcher.py:84: RuntimeWarning: divide by zero encountered in true_divide
c:\users\bver\miniconda3\envs\ayo\lib\site-packages\ipykernel_launcher.py:89: RuntimeWarning: invalid value encountered in arccos


took 8.292327 seconds.
took 8.183046 seconds.


...

took 8.414896 seconds.
took 9.354421 seconds.
took 8.895554 seconds.
took 9.253102 seconds.
took 9.273246 seconds.
took 8.333381 seconds.
took 8.217519 seconds.
took 8.158248 seconds.
took 8.202195 seconds.
took 8.237428 seconds.
took 8.234324 seconds.
took 8.303457 seconds.
took 8.260896 seconds.
took 8.338153 seconds.
took 8.139220 seconds.
took 8.295140 seconds.
took 8.433730 seconds.
took 8.633536 seconds.
took 8.482526 seconds.
took 8.327471 seconds.
took 8.292113 seconds.
took 8.365373 seconds.
took 8.335063 seconds.
took 8.336492 seconds.
took 8.386071 seconds.
took 8.311781 seconds.
took 8.363822 seconds.
took 8.343731 seconds.
took 8.291301 seconds.
took 8.353047 seconds.
took 8.303285 seconds.
took 8.416353 seconds.
took 8.315416 seconds.
took 8.299778 seconds.
took 8.410317 seconds.
took 8.186488 seconds.
took 8.199632 seconds.
took 8.377126 seconds.
took 8.341571 seconds.
took 8.422655 seconds.
took 8.524135 seconds.
took 8.485869 seconds.
took 8.337132 seconds.
took 8.3618

c:\users\bver\miniconda3\envs\ayo\lib\site-packages\ipykernel_launcher.py:84: RuntimeWarning: divide by zero encountered in true_divide
c:\users\bver\miniconda3\envs\ayo\lib\site-packages\ipykernel_launcher.py:89: RuntimeWarning: invalid value encountered in arccos


took 9.315703 seconds.


c:\users\bver\miniconda3\envs\ayo\lib\site-packages\ipykernel_launcher.py:84: RuntimeWarning: invalid value encountered in true_divide


took 9.384400 seconds.
took 9.299233 seconds.
took 8.728179 seconds.
took 8.595704 seconds.
